In [1]:
!nvidia-smi

Fri Nov 15 05:33:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import models, transforms
from PIL import Image
import os
import numpy as np

In [25]:
!pip install rarfile
import rarfile
rar_path = "/content/UTKFace.rar"
extract_path = "/content/UTKFace"
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

with rarfile.RarFile(rar_path) as rf:
    rf.extractall(extract_path)

# Confirm extraction by listing files
print("Files extracted:")
for root, dirs, files in os.walk(extract_path):
    for file in files:
        print(os.path.join(root, file))

Streaming output truncated to the last 5000 lines.
/content/UTKFace/UTKFace/36_1_3_20170119202144381.jpg.chip.jpg
/content/UTKFace/UTKFace/38_0_0_20170117120656921.jpg.chip.jpg
/content/UTKFace/UTKFace/30_1_1_20170114031528104.jpg.chip.jpg
/content/UTKFace/UTKFace/1_0_0_20170110213043946.jpg.chip.jpg
/content/UTKFace/UTKFace/1_0_0_20170109194450082.jpg.chip.jpg
/content/UTKFace/UTKFace/51_1_2_20170109131917693.jpg.chip.jpg
/content/UTKFace/UTKFace/78_1_0_20170120140748041.jpg.chip.jpg
/content/UTKFace/UTKFace/64_1_0_20170110160643892.jpg.chip.jpg
/content/UTKFace/UTKFace/36_0_0_20170104203841107.jpg.chip.jpg
/content/UTKFace/UTKFace/28_1_0_20170117121841046.jpg.chip.jpg
/content/UTKFace/UTKFace/26_0_0_20170116205259187.jpg.chip.jpg
/content/UTKFace/UTKFace/24_1_3_20170104235106686.jpg.chip.jpg
/content/UTKFace/UTKFace/26_1_1_20170116222929223.jpg.chip.jpg
/content/UTKFace/UTKFace/32_0_0_20170116002349487.jpg.chip.jpg
/content/UTKFace/UTKFace/1_1_2_20161219154531437.jpg.chip.jpg
/conten

In [54]:
# Assuming you're running on Google Colab and the device is CUDA-enabled (GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the pre-trained ResNet18 model and modify it
model = models.resnet18(weights='IMAGENET1K_V1')  # Pre-trained weights for ResNet18
model.fc = nn.Linear(model.fc.in_features, 1)  # Final layer changed to output a single scalar (age)

# Move the model to the GPU or CPU
model = model.to(device)



In [55]:
# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()  # Using Mean Squared Error (MSE) loss for regression

In [56]:
# Custom dataset class for the UTKFace dataset
class AgeDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder)]
        self.labels = [int(fname.split('_')[0]) for fname in os.listdir(image_folder)]  # Extracting the age from the filename

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [57]:
# Define the necessary image transformations for training
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to fit the input size of ResNet18
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizing with ImageNet's mean and std
])


In [58]:
# Load your dataset (adjust path accordingly)
train_dataset = AgeDataset(image_folder='/content/UTKFace/UTKFace', transform=transform)  # Adjust path
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Function to train the model
def train_model(model, train_loader, optimizer, criterion, device, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)

            # Flatten the output to match the shape of labels (single scalar per image)
            outputs = outputs.view(-1)  # Ensure output is of shape (batch_size,)

            # Calculate the loss
            loss = criterion(outputs, labels.float())  # Ensure labels are float for regression
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

# Train the model
train_model(model, train_loader, optimizer, criterion, device, epochs=10)

Epoch 1/10, Loss: 148.36258338691414
Epoch 2/10, Loss: 77.63736128967945
Epoch 3/10, Loss: 62.13957300199027
Epoch 4/10, Loss: 52.32508102859724
Epoch 5/10, Loss: 43.13287541032964
Epoch 6/10, Loss: 36.01973243538345
Epoch 7/10, Loss: 29.415419793482894
Epoch 8/10, Loss: 24.382072143709127
Epoch 9/10, Loss: 19.216762654050964
Epoch 10/10, Loss: 15.836726745934943


In [59]:
# Function to test the model
def test_model(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    with torch.no_grad():  # No need to compute gradients for validation
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            outputs = outputs.view(-1)  # Flatten output

            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate the accuracy or error (e.g., Mean Absolute Error)
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    mae = np.mean(np.abs(all_preds - all_labels))  # Mean Absolute Error
    print(f'Mean Absolute Error on test set: {mae}')

# Assuming you have the test dataset in the same format, define the test dataset and loader
test_dataset = AgeDataset(image_folder='/content/UTKFace/UTKFace', transform=transform)  # Adjust path
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Test the model
test_model(model, test_loader, device)


Mean Absolute Error on test set: 2.8667390906170693


In [60]:
import torch

# Function to calculate accuracy within a given threshold
def accuracy_within_threshold(preds, labels, threshold=5):
    # Calculate the absolute error
    abs_error = torch.abs(preds - labels)

    # Count how many predictions are within the threshold
    correct = (abs_error <= threshold).sum().item()

    # Calculate accuracy as the percentage of correct predictions
    accuracy = correct / len(labels)
    return accuracy

# Assuming `model`, `test_loader`, and `device` are already defined
model.eval()  # Set model to evaluation mode

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)


        preds = outputs.squeeze()  # Remove unnecessary dimensions

        all_preds.append(preds)
        all_labels.append(labels)

# Flatten the list of predictions and labels
all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)

# Calculate accuracy within a threshold of 5 years
accuracy = accuracy_within_threshold(all_preds, all_labels, threshold=5)

print(f'Accuracy within 5 years: {accuracy * 100:.2f}%')

Accuracy within 5 years: 83.58%


In [61]:
# Save the model state dict
model_save_path = '/content/age_detection_model.pth'  # Change path as needed
torch.save(model.state_dict(), model_save_path)

print(f'Model saved to {model_save_path}')


Model saved to /content/age_detection_model.pth


In [62]:
# Save the model state dict
model_save_path = '/content/age_detection_model.pkl'  # Change path as needed
torch.save(model.state_dict(), model_save_path)

print(f'Model saved to {model_save_path}')



Model saved to /content/age_detection_model.pkl


In [64]:
import pickle

# Save the model as a .pkl file
model_save_path = '/content/age_detectioon_model.pkl'  # Change path as needed
with open(model_save_path, 'wb') as f:
    pickle.dump(model, f)

print(f'Model saved to {model_save_path}')

Model saved to /content/age_detectioon_model.pkl


In [65]:
#done